In [ ]:
from snowflake.snowpark import Session
import snowflake.connector
import logging
import requests
import json
import configparser
import sys, os
import warnings
import base64
import glob
import pandas as pd  # Make sure to import pandas

# Setup and cleanup
sys.path.append(os.path.realpath('..'))
warnings.filterwarnings("ignore")
os.system('cls' if os.name == 'nt' else 'clear')
masterdir = os.getcwd()
print(masterdir)

# Configuration
config = configparser.ConfigParser()
config.read(os.path.join(masterdir, '03-configfiles', 'pw-config.ini'))

acct = config.get('snowflake', 'acct')
user = config.get('snowflake', 'user')
pwd = config.get('snowflake', 'password')
role = config.get('snowflake', 'role')
wh = config.get('snowflake', 'warehouse')
db = config.get('snowflake', 'database')
sch = config.get('snowflake', 'schema')

print(acct)

try:
    # Configure a connection
    conn = snowflake.connector.connect(
        user=user,
        password=pwd,
        account=acct,
        warehouse=wh,
        database=db,
        schema=sch
    )

    # Create a cursor object
    cur = conn.cursor()
    
    # Execute the query
    cur.execute("SELECT * FROM surveydata")

    # Fetch all the results
    results = cur.fetchall()

    # Convert results into a DataFrame
    # Assuming `cur.description` contains the column names
    column_names = [col[0] for col in cur.description]
    df = pd.DataFrame(results, columns=column_names)

    # Display the DataFrame
    print(df.head())  # This will print the first few rows of the DataFrame

finally:
    # Make sure to close the cursor and connection when done
    if 'cur' in locals():
        cur.close()
    if 'conn' in locals():
        conn.close()
